In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import NarrowbandMetadata
from torchsig.signals.builders.constellation import ConstellationSignalBuilder
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
from torchsig.utils.dsp import (
    torchsig_complex_data_type,
    frequency_shift,
    multistage_polyphase_resampler
)

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 10, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal.

    """
    sample_rate = 10e6
    md = NarrowbandMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['qpsk'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ConstellationSignalBuilder(
        dataset_metadata = md, 
        class_name = 'qpsk',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale).astype(torchsig_complex_data_type)

    return signal

In [ ]:
def generate_tone_signal(num_iq_samples: int = 10, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband tone Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal.

    """
    sample_rate = 10e6
    md = NarrowbandMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['tone'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ToneSignalBuilder(
        dataset_metadata = md, 
        class_name = 'tone',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale).astype(torchsig_complex_data_type)

    return signal

In [ ]:
# test data
N = 1024
qpsk_bb_data = generate_qpsk_signal(num_iq_samples = N, scale = 1.0).data
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data

# upsample, rescale, then frequency shift
qpsk_8x_data = multistage_polyphase_resampler(qpsk_bb_data, 8.0)
tone_8x_data = multistage_polyphase_resampler(tone_bb_data, 8.0)

qpsk_data = frequency_shift(qpsk_8x_data, 0.125, 1.0)
qpsk_data = qpsk_data / np.max(np.abs(qpsk_data))

tone1 = frequency_shift(tone_8x_data, 0.10, 1.0)
tone1 = tone1 / np.max(np.abs(tone1)) 
tone2 = frequency_shift(tone_8x_data, 0.15, 1.0)
tone2 = tone2 / np.max(np.abs(tone2))
two_tone_data = tone1 + tone2

freq_vec = np.arange(-1.0/2,1.0/2,1.0/(N*8))

In [ ]:
# third-order IMD model
coeffs = np.array([1.0,0.0,1.0])
qpsk_nl_data = F.intermodulation_products(data = qpsk_data, coeffs = coeffs)
two_tone_nl_data = F.intermodulation_products(data = two_tone_data, coeffs = coeffs)

Q = np.fft.fftshift(np.fft.fft(qpsk_data))/(N*8);
Q_nl = np.fft.fftshift(np.fft.fft(qpsk_nl_data))/(N*8);
T = np.fft.fftshift(np.fft.fft(two_tone_data))/(N*8);
T_nl = np.fft.fftshift(np.fft.fft(two_tone_nl_data))/(N*8);

low_tone_ind = int((0.10 + 0.5)*(1024*8))
high_im_ind = int((0.20 + 0.5)*(1024*8))                
print(low_tone_ind, np.abs(T_nl[low_tone_ind]))
print(high_im_ind, np.abs(T_nl[high_im_ind]))
print(np.abs(T_nl[low_tone_ind]) / np.abs(T_nl[high_im_ind]))

             
fig, ax = plt.subplots(2,1, figsize=(8, 8))
ax[0].plot(freq_vec, 10*np.log10(np.abs(Q_nl*Q_nl)), 'b');
ax[0].plot(freq_vec, 10*np.log10(np.abs(Q*Q)), 'k-');
ax[0].set_ylim([-100, -20])
ax[0].set_ylabel('Magnitude (log10)',fontsize='large');
ax[0].set_title('Third-order Nonlinearity Model: QPSK')
ax[0].legend(['Nonlinear','Linear'],fontsize='large', loc='upper left');
ax[1].plot(freq_vec, 10*np.log10(np.abs(T_nl*T_nl)), 'b');
ax[1].plot(freq_vec, 10*np.log10(np.abs(T*T)), 'k-');
ax[1].set_ylim([-80, 5]);
ax[1].set_xlabel('Frequency (Fs norm)',fontsize='large');
ax[1].set_xlabel('Frequency (Fs norm)',fontsize='large');
ax[1].set_ylabel('Magnitude (log10)',fontsize='large');
ax[1].set_title('Third-order Nonlinearity Model: Two Tones (0.1, 0.15)')
ax[1].legend(['Nonlinear','Linear'],fontsize='large', loc='upper left');

In [ ]:
# fifth-order IMD model
coeffs = np.array([1.0,0.0,1.0,0.0,1.0])
qpsk_nl_data = F.intermodulation_products(data = qpsk_data, coeffs = coeffs)
two_tone_nl_data = F.intermodulation_products(data = two_tone_data, coeffs = coeffs)

Q = np.fft.fftshift(np.fft.fft(qpsk_data))/(N*8);
Q_nl = np.fft.fftshift(np.fft.fft(qpsk_nl_data))/(N*8);
T = np.fft.fftshift(np.fft.fft(two_tone_data))/(N*8);
T_nl = np.fft.fftshift(np.fft.fft(two_tone_nl_data))/(N*8);

fig, ax = plt.subplots(2,1, figsize=(8, 8))
ax[0].plot(freq_vec, 10*np.log10(np.abs(Q_nl*Q_nl)), 'b');
ax[0].plot(freq_vec, 10*np.log10(np.abs(Q*Q)), 'k-');
ax[0].set_ylim([-100, -20]);
ax[0].set_ylabel('Magnitude (log10)',fontsize='large');
ax[0].set_title('Fifth-order Nonlinearity Model: QPSK');
ax[0].legend(['Nonlinear','Linear'],fontsize='large', loc='upper left');
ax[1].plot(freq_vec, 10*np.log10(np.abs(T_nl*T_nl)), 'b');
ax[1].plot(freq_vec, 10*np.log10(np.abs(T*T)), 'k-');
ax[1].set_ylim([-80, 10]);
ax[1].set_xlabel('Frequency (Fs norm)',fontsize='large');
ax[0].set_ylabel('Magnitude (log10)',fontsize='large');
ax[1].set_title('Fifth-order Nonlinearity Model: Two Tones (0.1, 0.15)');
ax[1].legend(['Nonlinear','Linear'],fontsize='large', loc='upper left');

In [ ]:
# Examine Amplifier Response: Two Tone Signal

# baseline test data
N = 1024
freq_vec = np.arange(-1.0/2,1.0/2,1.0/(N*8))

tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data
tone_8x_data = multistage_polyphase_resampler(tone_bb_data, 8.0)
tone1 = frequency_shift(tone_8x_data, 0.10, 1.0)
tone1 = tone1 / np.max(np.abs(tone1)) 
tone2 = frequency_shift(tone_8x_data, 0.15, 1.0)
tone2 = tone2 / np.max(np.abs(tone2))
two_tone_data = tone1 + tone2

# nonlinear amplifier model

Pin  = 10**((np.array([-20., -10.,  0.,  5., 10. ])) / 10)
#Pin = np.arange(0.0, 10.0, 0.1)

Pout = 10**((np.array([-10.,   0.,  9., 9.9, 10. ])) / 10)
# Pout  = 10**((np.array([-25., -10.,  0.,  5., 10. ])) / 10)
# Pout = Pin
# Pout[-90:] = Pout[-90]

Phi  = np.deg2rad(np.array([-2., -4., 7., 12., 23.]))        
#Phi = np.zeros_like(Pin)

# test
data_in = two_tone_data
data_out = F.nonlinear_amplifier_am_pm(
    data = data_in,
    Pin  = Pin,
    Pout = Pout,
    Phi  = Phi
)

input_mag = np.abs(data_in)
input_power = np.abs(data_in)**2
output_mag = np.abs(data_out)
output_power = np.abs(data_out)**2

input_phase_rad = np.angle(data_in)
output_phase_rad = np.angle(data_out)
phase_diff_rad = np.unwrap(output_phase_rad - input_phase_rad)
phase_diff_degree = np.rad2deg(phase_diff_rad)

### Plots
# AM/AM Power
fig = plt.figure(figsize=(8, 8))
plt.plot(Pin,Pout,'.-');
plt.plot(input_power,output_power,'k.');
plt.title('Nonlinear Amplifier Model AM/AM: Two Tone Test Signal',fontsize='large');
plt.xlabel('Pin',fontsize='large');
plt.ylabel('Pout',fontsize='large');
plt.legend(['Model','Test'],fontsize='large');

# AM/PM Phase
fig = plt.figure(figsize=(8, 8))
plt.plot(Pin,Phi,'.-');
plt.plot(input_power,phase_diff_rad,'k.');
plt.title('Nonlinear Amplifier Model AM/PM: Two Tone Test Signal',fontsize='large');
plt.xlabel('Pin',fontsize='large');
plt.ylabel('Phi',fontsize='large');
plt.legend(['Model','Test'],fontsize='large');

# Spectrum
fig = plt.figure(figsize=(8, 8))
T = np.fft.fftshift(np.fft.fft(data_in))/(N*8);
T_nl = np.fft.fftshift(np.fft.fft(data_out))/(N*8);

plt.plot(freq_vec, 10*np.log10(np.abs(T_nl*T_nl)), 'b');
plt.plot(freq_vec, 10*np.log10(np.abs(T*T)), 'k-');
plt.ylim([-80, 10]);
plt.xlabel('Frequency (Fs norm)',fontsize='large');
plt.ylabel('Magnitude (log10)',fontsize='large');
plt.title('Fifth-order Nonlinearity Model: Two Tones (0.1, 0.15)');
plt.legend(['Nonlinear','Linear'],fontsize='large', loc='upper left');

In [ ]:
# Examine Amplifier Response: QPSK Signal

# baseline test data
N = 1024
freq_vec = np.arange(-1.0/2,1.0/2,1.0/(N*8))

qpsk_bb_data = generate_qpsk_signal(num_iq_samples = N, scale = 1.0).data
qpsk_8x_data = multistage_polyphase_resampler(qpsk_bb_data, 8.0)
qpsk_data = frequency_shift(qpsk_8x_data, 0.125, 1.0)
qpsk_data = 2 * qpsk_data / np.max(np.abs(qpsk_data))

# nonlinear amplifier model

Pin  = 10**((np.array([-20., -10.,  0.,  5., 10. ])) / 10)
#Pin = np.arange(0.0, 10.0, 0.1)

Pout = 10**((np.array([-10.,   0.,  9., 9.9, 10. ])) / 10)
# Pout  = 10**((np.array([-25., -10.,  0.,  5., 10. ])) / 10)
# Pout = Pin
# Pout[-90:] = Pout[-90]

Phi  = np.deg2rad(np.array([-2., -4., 7., 12., 23.]))        
#Phi = np.zeros_like(Pin)

# test
data_in = qpsk_data
data_out = F.nonlinear_amplifier_am_pm(
    data = data_in,
    Pin  = Pin,
    Pout = Pout,
    Phi  = Phi
)

input_mag = np.abs(data_in)
input_power = np.abs(data_in)**2
output_mag = np.abs(data_out)
output_power = np.abs(data_out)**2

input_phase_rad = np.angle(data_in)
output_phase_rad = np.angle(data_out)
phase_diff_rad = np.unwrap(output_phase_rad - input_phase_rad)
phase_diff_degree = np.rad2deg(phase_diff_rad)

# print(Pin)
# print(Pout)
# print(np.mean(input_power), np.max(input_power))

### Plots
# AM/AM Power
fig = plt.figure(figsize=(8, 8))
plt.plot(Pin,Pout,'.-');
plt.plot(input_power,output_power,'k.');
plt.title('Nonlinear Amplifier Model AM/AM: QPSK Test Signal',fontsize='large');
plt.xlabel('Pin',fontsize='large');
plt.ylabel('Pout',fontsize='large');
plt.legend(['Model','Test'],fontsize='large');

# AM/PM Phase
fig = plt.figure(figsize=(8, 8))
plt.plot(Pin,Phi,'.-');
plt.plot(input_power,phase_diff_rad,'k.');
plt.title('Nonlinear Amplifier Model AM/PM: QPSK Test Signal',fontsize='large');
plt.xlabel('Pin',fontsize='large');
plt.ylabel('Phi',fontsize='large');
plt.legend(['Model','Test'],fontsize='large');

# Spectrum
fig = plt.figure(figsize=(8, 8))
T = np.fft.fftshift(np.fft.fft(data_in))/(N*8);
T_nl = np.fft.fftshift(np.fft.fft(data_out))/(N*8);

plt.plot(freq_vec, 10*np.log10(np.abs(T_nl*T_nl)), 'b');
plt.plot(freq_vec, 10*np.log10(np.abs(T*T)), 'k-');
plt.ylim([-80, 10]);
plt.xlabel('Frequency (Fs norm)',fontsize='large');
plt.ylabel('Magnitude (log10)',fontsize='large');
plt.title('Fifth-order Nonlinearity Model: QPSK');
plt.legend(['Nonlinear','Linear'],fontsize='large', loc='upper left');